In [ ]:
'''
nominal var has influence on the scale variable

nominal var is school location

scale var is the marks of students in the exam
'''

In [ ]:
import pandas
import numpy
import matplotlib.pyplot as plt

In [ ]:
mydata = pandas.read_csv('/content/StudentStatistics.csv', sep=';')
mydata

In [ ]:
'Location'
'Over_Grade'

In [ ]:
mydata.groupby('Location')['Over_Grade'].mean()

In [ ]:
mydata.groupby('Location')['Over_Grade'].std()

In [ ]:
mydata.groupby('Location')['Over_Grade'].agg(['mean', 'median', 'std', 'min', 'max'])

In [ ]:
rotterdamScores = mydata['Over_Grade'][mydata['Location']=='Rotterdam'].dropna()
diemenScores = mydata['Over_Grade'][mydata['Location']=='Diemen'].dropna()
haarlemScores = mydata['Over_Grade'][mydata['Location']=='Haarlem'].dropna()

In [ ]:
plt.figure(1, figsize=(10,10))

plt.subplot(3,1,1)
plt.hist(rotterdamScores, color='skyblue', edgecolor='green', lw=3, bins = [10, 20, 30, 40, 50, 60, 70, 80, 90, 100])
plt.xlabel('Rotterdam')
plt.ylabel('Frequency')

plt.subplot(3,1,2)
plt.hist(diemenScores, color='skyblue', edgecolor='green', lw=3, bins = [10, 20, 30, 40, 50, 60, 70, 80, 90, 100])
plt.xlabel('Diemen')
plt.ylabel('Frequency')


plt.subplot(3,1,3)
plt.hist(haarlemScores, color='skyblue', edgecolor='green', lw=3, bins = [10, 20, 30, 40, 50, 60, 70, 80, 90, 100])
plt.xlabel('Haarlem')
plt.ylabel('Frequency')
plt.show()

#Welch ANOVA

This is a test that is often used to determine if there are differences in means between different categories.

The Welch test for means is often used if the variances within the groups are not equal. There are other tests though (Brown-Forsythe, James, Alexander-Govern, etc) that also do this. Delacre et al. (2019) recommend to use the Welch ANOVA instead of the classic and Brown-Forsythe versions, but there are many articles on the topic of which test to use and each with slightly different recommendations.  

The test is actually a modification of the Cochran test for means (separate file on that). The test is also sometimes referred to as the Aspin-Welch test, to credit besides Welch (1947, 1951), also Aspin (Aspin, 1948; Aspin & Welch, 1949).

There a few different ways you can do this test with Python. If you have a pandas dataframe, the easiest method (in my opinion) is by using the Pingouin package or the statsmodels.api. You can simply let it know which fields are involved and basically you're done.

I'll show an example for both, and in the appendix how you can even avoid almost entirely packages by going over the formulas involved (it still needs a package for the F-distribution, which would go too far to also do without a package).

Lets get started.

In [ ]:
'''
H0 - there is no influence of student location on the marks
Ha - there is influence of student location on the marks

'''

In [ ]:
!pip install pingouin

In [ ]:
import pingouin as pin

In [ ]:
pin.welch_anova(dv='Over_Grade', between='Location', data=mydata)

In [ ]:
#null hypothesis is rejected and test is significant
pvalue = 0.000536

The most commonly reported effect size that goes with a one-way ANOVA is eta-squared (η<sup>2</sup>) (Bakeman, 2005; Levine & Hullett, 2002) although omega squared (ω<sup>2</sup>) might actually be preferred.

An eta square of 0 would mean no differences (and no influence), while 1 would indicate a full dependency. Unfortunately there is no formal way to determine if a specific value is high or low, and the rules of thumb floating around on the internet vary quite a lot, often depending on the field (e.g. biology, medicine, business, etc.).  One of them is from Cohen (1988):

|η<sup>2</sup>|Interpretation|
|-------|---------------|
|0.00 < 0.01|Negligible|
|0.01 < 0.06|Small|
|0.06 < 0.14|Medium|
|0.14 <= 1.00|Large|

*Note that Cohen actually uses another effect size (Cohen's f) for which he reports the rules of thumb on pages 285-287, on page 283 he has a formula to convert f to eta-squared. I've converted the threholds using that formula.*

Lets find out on how to determine eta-squared with Python.

In [ ]:
#np2 - partial eta square

etaSq = 0.263336

if etaSq < .01:
    qual = 'Negligible'
elif etaSq < .06:
    qual = 'Small'
elif etaSq < .14:
    qual = 'Medium'
else:
    qual = 'Large'

qual

In [ ]:
len(rotterdamScores) + len(haarlemScores) + len(diemenScores)

The Welch ANOVA showed that Location had significant large effect on how students get marks in the test, F(2, 48) = 10.195117, p = .001, η2 = .26.